In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [5]:
# train_solutions = json.loads("/kaggle/input/arc-prize-2025/arc-agi_training_solutions.json")
# train_challenges = json.loads("/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json")
# eval_solutions = json.loads("/kaggle/input/arc-prize-2025/arc-agi_evaluation_solutions.json")
# eval_challenges = json.loads("/kaggle/input/arc-prize-2025/arc-agi_evaluation_challenges.json")

# file_path
train_solutions = json.loads(open("data/ARC_Prize_2025/arc-agi_training_solutions.json").read())
train_challenges = json.loads(open("data/ARC_Prize_2025/arc-agi_training_challenges.json").read())
eval_solutions = json.loads(open("data/ARC_Prize_2025/arc-agi_evaluation_solutions.json").read())
eval_challenges = json.loads(open("data/ARC_Prize_2025/arc-agi_evaluation_challenges.json").read())

# print the length of each dataset
print(len(train_solutions), len(train_challenges), len(eval_solutions), len(eval_challenges))
print("head of train_solutions")
print(train_solutions["00576224"])
print("head of train_challenges")
print(train_challenges["00576224"])
print(train_challenges["00576224"].keys())
print(train_challenges["00576224"]["train"])
print(train_challenges["00576224"]["train"][0])
print(train_challenges["00576224"]["train"][0]["input"])
print(train_challenges["00576224"]["train"][0]["output"])
print(train_challenges["00576224"]["train"][0]["input"][0])

1000 1000 120 120
head of train_solutions
[[[3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8], [2, 3, 2, 3, 2, 3], [8, 7, 8, 7, 8, 7], [3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8]]]
head of train_challenges
{'train': [{'input': [[7, 9], [4, 3]], 'output': [[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7, 9, 7, 9, 7], [3, 4, 3, 4, 3, 4], [7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3]]}, {'input': [[8, 6], [6, 4]], 'output': [[8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4], [6, 8, 6, 8, 6, 8], [4, 6, 4, 6, 4, 6], [8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4]]}], 'test': [{'input': [[3, 2], [7, 8]]}]}
dict_keys(['train', 'test'])
[{'input': [[7, 9], [4, 3]], 'output': [[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7, 9, 7, 9, 7], [3, 4, 3, 4, 3, 4], [7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3]]}, {'input': [[8, 6], [6, 4]], 'output': [[8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4], [6, 8, 6, 8, 6, 8], [4, 6, 4, 6, 4, 6], [8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4]]}]
{'input': [[7, 9], [4, 3]], 'output': [[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7,

- Transformers
- Graph Neural Networks
- Program Synthesis / Sysbolic AI
- Neuro-Symbolic AI

1. First of all, CNN + Transformers is the base line.
2. Second of all, Vision Transformer + Attension.
3. Third of all, Graph Neural Network.
4. Last of all, Neuro-Symbolic like hybrid approach.

# Transformers

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0

        # Initialize dimensions
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query
        self.W_k = nn.Linear(d_model, d_model) # Key
        self.W_v = nn.Linear(d_model, d_model) # Value
        self.W_o = nn.Linear(d_model, d_model) # Output

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # Apply mask if provided
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)

        

In [7]:
for i, tensordict in enumerate(train_challenges):
    print(i, tensordict)
    print(train_challenges[tensordict])
    if i == 3:
        break

0 00576224
{'train': [{'input': [[7, 9], [4, 3]], 'output': [[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7, 9, 7, 9, 7], [3, 4, 3, 4, 3, 4], [7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3]]}, {'input': [[8, 6], [6, 4]], 'output': [[8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4], [6, 8, 6, 8, 6, 8], [4, 6, 4, 6, 4, 6], [8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4]]}], 'test': [{'input': [[3, 2], [7, 8]]}]}
1 007bbfb7
{'train': [{'input': [[6, 6, 0], [6, 0, 0], [0, 6, 6]], 'output': [[6, 6, 0, 6, 6, 0, 0, 0, 0], [6, 0, 0, 6, 0, 0, 0, 0, 0], [0, 6, 6, 0, 6, 6, 0, 0, 0], [6, 6, 0, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0], [0, 6, 6, 0, 0, 0, 0, 0, 0], [0, 0, 0, 6, 6, 0, 6, 6, 0], [0, 0, 0, 6, 0, 0, 6, 0, 0], [0, 0, 0, 0, 6, 6, 0, 6, 6]]}, {'input': [[4, 0, 4], [0, 0, 0], [0, 4, 0]], 'output': [[4, 0, 4, 0, 0, 0, 4, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 4, 0, 0, 0, 0, 0, 4, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 4, 0, 4, 0, 0, 0], [0, 0, 0, 0, 0, 

# Reference

- [1] https://www.datacamp.com/tutorial/building-a-transformer-with-py-torch
- [2] https://medium.com/data-science/transformers-explained-visually-part-3-multi-head-attention-deep-dive-1c1ff1024853
- [3] https://www.ibm.com/think/topics/positional-encoding